# Y2017M11D24_RH_Prepare_Image_Collections_EE_V01

* Purpose of script: put all earth engine imagecollections in the same format (millionm^3  and dimensionless)
* Kernel used: python27
* Date created: 20171124  

The imageCollections that need a unit conversion are : Discharge and Runoff. 



In [1]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

('Y2017M11D28', 'UTC 12:05')


'2.7.13 |Continuum Analytics, Inc.| (default, Dec 20 2016, 23:09:15) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]'

In [2]:
EE_PATH = "projects/WRI-Aquaduct/PCRGlobWB20V07"

AREA_IMAGE_FILE_NAME = "area_5min_m2V11"


SCRIPT_NAME = "Y2017M11D24_RH_Prepare_Image_Collections_EE_V01"

OUTPUT_VERSION = 1

# Unfortunately specifying the dimensions caused the script to crash (internal error on Google's side) Specify scale instead.

DIMENSION5MIN = {}
DIMENSION5MIN["x"] = 4320
DIMENSION5MIN["y"] = 2160
CRS = "EPSG:4326"

MAXPIXELS =1e10

icIds = ["projects/WRI-Aquaduct/PCRGlobWB20V07/global_historical_riverdischarge_month_m3second_5min_1960_2014",
        "projects/WRI-Aquaduct/PCRGlobWB20V07/global_historical_riverdischarge_year_m3second_5min_1960_2014",
        "projects/WRI-Aquaduct/PCRGlobWB20V07/global_historical_runoff_month_mmonth_5min_1958_2014",
        "projects/WRI-Aquaduct/PCRGlobWB20V07/global_historical_runoff_year_myear_5min_1958_2014"]


In [3]:
import ee
import re
import subprocess
import pandas as pd
from calendar import monthrange, isleap

In [4]:
ee.Initialize()

ICs not in right format: discharge (m^3 / s) and runoff (m/month or m/year)

In [5]:
sPerD = 86400 #seconds per day

In [6]:
areaImage = ee.Image("%s/%s"%(EE_PATH,AREA_IMAGE_FILE_NAME))

In [7]:
dimensions = "%sx%s" %(DIMENSION5MIN["x"],DIMENSION5MIN["y"])

In [8]:
crsTransform = [
                0.0833333309780367,
                0,
                -179.99999491255934,
                0,
                -0.0833333309780367,
                90.00000254430942
              ]

In [9]:
def newImageId(imageId):
    return re.sub('m3second|mmonth|myear',"millionm3",imageId)

def findUnit(imageId):
    if re.search("m3second",imageId):
        unit = "m3second"
    elif re.search("mmonth",imageId):
        unit = "mmonth"
    elif re.search("myear",imageId):
        unit = "myear"
    else:
        unit = "error"
    return unit


def toVolumeAndExport(row):
    unit = row["unit"]
    temporalResolution = row["image"].get("temporal_resolution").getInfo()
    year = int(row["image"].get("year").getInfo())
    month = int(row["image"].get("month").getInfo())


    
    if unit == "m3second":        
        if temporalResolution == "month":
            sPerMonth = monthrange(year,month)[1]*(86400)
            newImage = row["image"].multiply(sPerMonth).divide(1e6)
            
             
        elif temporalResolution == "year":
            daysPerYear = 365 if isleap(2005) else 366
            sPerYear = daysPerYear*(86400)
            newImage = row["image"].multiply(sPerYear).divide(1e6)
            
        else:
            print("error",row)  
        
    elif unit == "mmonth" or unit == "myear":
        newImage = row["image"].multiply(areaImage).divide(1e6)
    else:
        pass
    
    newImage = newImage.copyProperties(row["image"])
    newImage = newImage.set("units","millionm3")
    newImage = newImage.set("script_used",SCRIPT_NAME)
    newImage = newImage.set("version",OUTPUT_VERSION)
    
    description = "%sV%0.2d" %(row["image"].get("exportdescription").getInfo(),OUTPUT_VERSION)
    print(description)
    
    task = ee.batch.Export.image.toAsset(
        image =  ee.Image(newImage),
        description = description,
        assetId = row["newImageId"],
        dimensions = dimensions,
        crs = CRS,
        crsTransform = crsTransform,
        maxPixels = MAXPIXELS     
    )
    task.start()       
    dfOut.at[index,"newImage"] = newImage
    

In [10]:
dfIcs = pd.DataFrame()
dfIcs["icId"] = icIds

### Creating new ImageCollections

In [11]:
dfIcs["newIcId"] = dfIcs["icId"].apply(newImageId)

In [12]:
for index, row in dfIcs.iterrows():
    command = "earthengine create collection %s" %row["newIcId"]
    result = subprocess.check_output(command,shell=True)
    print(command,result)

('earthengine create collection projects/WRI-Aquaduct/PCRGlobWB20V07/global_historical_riverdischarge_month_millionm3_5min_1960_2014', 'Asset projects/WRI-Aquaduct/PCRGlobWB20V07/global_historical_riverdischarge_month_millionm3_5min_1960_2014 already exists\n')
('earthengine create collection projects/WRI-Aquaduct/PCRGlobWB20V07/global_historical_riverdischarge_year_millionm3_5min_1960_2014', 'Asset projects/WRI-Aquaduct/PCRGlobWB20V07/global_historical_riverdischarge_year_millionm3_5min_1960_2014 already exists\n')
('earthengine create collection projects/WRI-Aquaduct/PCRGlobWB20V07/global_historical_runoff_month_millionm3_5min_1958_2014', 'Asset projects/WRI-Aquaduct/PCRGlobWB20V07/global_historical_runoff_month_millionm3_5min_1958_2014 already exists\n')
('earthengine create collection projects/WRI-Aquaduct/PCRGlobWB20V07/global_historical_runoff_year_millionm3_5min_1958_2014', 'Asset projects/WRI-Aquaduct/PCRGlobWB20V07/global_historical_runoff_year_millionm3_5min_1958_2014 already

In [13]:
dfImages2 = pd.DataFrame()

for index, row in dfIcs.iterrows():
    command = "earthengine ls %s" %row["icId"]
    assetList = subprocess.check_output(command,shell=True).splitlines()
    dfImages = pd.DataFrame(assetList)
    dfImages.columns = ["imageId"]
    dfImages["icId"] = row["icId"]
    dfImages2= dfImages2.append(dfImages)

In [14]:
dfImages2["newImageId"] = dfImages2["imageId"].apply(newImageId)
dfImages2["newIcId"] = dfImages2["icId"].apply(newImageId)
dfImages2["unit"] = dfImages2["imageId"].apply(findUnit)
dfImages2["image"] = dfImages2["imageId"].apply(lambda x: ee.Image(x))
dfImages2 = dfImages2.set_index("imageId",drop=False)

In [15]:
dfOut = dfImages2.copy()

In [16]:
i = 0
errorlog = []
startLoop = datetime.datetime.now()
for index, row in dfImages2.iterrows():
    i += 1 
    try:
        toVolumeAndExport(row)
    except:
        errorlog.append(index)
    elapsed = datetime.datetime.now() - startLoop
    print(i,dfImages2.shape[0])
    print(elapsed)
    
    

riverdischarge_monthY1960M01V01
(1, 1456)
0:00:04.235106
riverdischarge_monthY1960M02V01
(2, 1456)
0:00:08.001637
riverdischarge_monthY1960M03V01
(3, 1456)
0:00:12.252586
riverdischarge_monthY1960M04V01
(4, 1456)
0:00:16.135188
riverdischarge_monthY1960M05V01
(5, 1456)
0:00:20.400431
riverdischarge_monthY1960M06V01
(6, 1456)
0:00:24.688635
riverdischarge_monthY1960M07V01
(7, 1456)
0:00:28.962729
riverdischarge_monthY1960M08V01
(8, 1456)
0:00:33.278462
riverdischarge_monthY1960M09V01
(9, 1456)
0:00:37.976822
riverdischarge_monthY1960M10V01
(10, 1456)
0:00:42.710155
riverdischarge_monthY1960M11V01
(11, 1456)
0:00:47.445628
riverdischarge_monthY1960M12V01
(12, 1456)
0:00:52.098980
riverdischarge_monthY1961M01V01
(13, 1456)
0:00:56.816575
riverdischarge_monthY1961M02V01
(14, 1456)
0:01:01.631052
riverdischarge_monthY1961M03V01
(15, 1456)
0:01:06.296174
riverdischarge_monthY1961M04V01
(16, 1456)
0:01:10.981944
riverdischarge_monthY1961M05V01
(17, 1456)
0:01:15.612521
riverdischarge_monthY19

riverdischarge_monthY1971M10V01
(142, 1456)
0:10:14.229211
riverdischarge_monthY1971M11V01
(143, 1456)
0:10:18.474419
riverdischarge_monthY1971M12V01
(144, 1456)
0:10:22.225315
riverdischarge_monthY1972M01V01
(145, 1456)
0:10:26.511654
riverdischarge_monthY1972M02V01
(146, 1456)
0:10:30.452335
riverdischarge_monthY1972M03V01
(147, 1456)
0:10:34.724086
riverdischarge_monthY1972M04V01
(148, 1456)
0:10:39.008632
riverdischarge_monthY1972M05V01
(149, 1456)
0:10:42.855273
riverdischarge_monthY1972M06V01
(150, 1456)
0:10:46.554346
riverdischarge_monthY1972M07V01
(151, 1456)
0:10:50.786587
riverdischarge_monthY1972M08V01
(152, 1456)
0:10:55.052270
riverdischarge_monthY1972M09V01
(153, 1456)
0:10:59.327573
riverdischarge_monthY1972M10V01
(154, 1456)
0:11:03.065576
riverdischarge_monthY1972M11V01
(155, 1456)
0:11:07.318358
riverdischarge_monthY1972M12V01
(156, 1456)
0:11:11.574393
riverdischarge_monthY1973M01V01
(157, 1456)
0:11:15.354130
riverdischarge_monthY1973M02V01
(158, 1456)
0:11:19.6502

riverdischarge_monthY1983M05V01
(281, 1456)
0:19:59.577975
riverdischarge_monthY1983M06V01
(282, 1456)
0:20:04.319968
riverdischarge_monthY1983M07V01
(283, 1456)
0:20:09.037469
riverdischarge_monthY1983M08V01
(284, 1456)
0:20:13.676411
riverdischarge_monthY1983M09V01
(285, 1456)
0:20:17.939963
riverdischarge_monthY1983M10V01
(286, 1456)
0:20:22.692911
riverdischarge_monthY1983M11V01
(287, 1456)
0:20:26.440822
riverdischarge_monthY1983M12V01
(288, 1456)
0:20:31.126928
riverdischarge_monthY1984M01V01
(289, 1456)
0:20:35.417813
riverdischarge_monthY1984M02V01
(290, 1456)
0:20:39.642490
riverdischarge_monthY1984M03V01
(291, 1456)
0:20:43.849809
riverdischarge_monthY1984M04V01
(292, 1456)
0:20:48.136123
riverdischarge_monthY1984M05V01
(293, 1456)
0:20:52.382161
riverdischarge_monthY1984M06V01
(294, 1456)
0:20:56.598733
riverdischarge_monthY1984M07V01
(295, 1456)
0:21:00.921185
riverdischarge_monthY1984M08V01
(296, 1456)
0:21:05.139475
riverdischarge_monthY1984M09V01
(297, 1456)
0:21:09.4669

riverdischarge_monthY1994M12V01
(420, 1456)
0:30:02.142440
riverdischarge_monthY1995M01V01
(421, 1456)
0:30:06.262458
riverdischarge_monthY1995M02V01
(422, 1456)
0:30:10.389947
riverdischarge_monthY1995M03V01
(423, 1456)
0:30:14.640304
riverdischarge_monthY1995M04V01
(424, 1456)
0:30:18.797502
riverdischarge_monthY1995M05V01
(425, 1456)
0:30:22.969073
riverdischarge_monthY1995M06V01
(426, 1456)
0:30:26.663062
riverdischarge_monthY1995M07V01
(427, 1456)
0:30:30.881264
riverdischarge_monthY1995M08V01
(428, 1456)
0:30:35.095592
riverdischarge_monthY1995M09V01
(429, 1456)
0:30:38.805489
riverdischarge_monthY1995M10V01
(430, 1456)
0:30:43.002236
riverdischarge_monthY1995M11V01
(431, 1456)
0:30:47.101677
riverdischarge_monthY1995M12V01
(432, 1456)
0:30:51.265509
riverdischarge_monthY1996M01V01
(433, 1456)
0:30:55.437028
riverdischarge_monthY1996M02V01
(434, 1456)
0:30:59.557765
riverdischarge_monthY1996M03V01
(435, 1456)
0:31:03.698864
riverdischarge_monthY1996M04V01
(436, 1456)
0:31:07.9344

riverdischarge_monthY2006M07V01
(559, 1456)
0:39:49.278187
riverdischarge_monthY2006M08V01
(560, 1456)
0:39:52.973277
riverdischarge_monthY2006M09V01
(561, 1456)
0:39:57.316125
riverdischarge_monthY2006M10V01
(562, 1456)
0:40:00.617639
riverdischarge_monthY2006M11V01
(563, 1456)
0:40:04.381536
riverdischarge_monthY2006M12V01
(564, 1456)
0:40:08.621403
riverdischarge_monthY2007M01V01
(565, 1456)
0:40:12.743446
riverdischarge_monthY2007M02V01
(566, 1456)
0:40:16.889102
riverdischarge_monthY2007M03V01
(567, 1456)
0:40:20.617671
riverdischarge_monthY2007M04V01
(568, 1456)
0:40:24.734025
riverdischarge_monthY2007M05V01
(569, 1456)
0:40:28.962457
riverdischarge_monthY2007M06V01
(570, 1456)
0:40:33.222711
riverdischarge_monthY2007M07V01
(571, 1456)
0:40:36.920360
riverdischarge_monthY2007M08V01
(572, 1456)
0:40:40.596388
riverdischarge_monthY2007M09V01
(573, 1456)
0:40:44.883937
riverdischarge_monthY2007M10V01
(574, 1456)
0:40:48.548865
riverdischarge_monthY2007M11V01
(575, 1456)
0:40:52.2545

(698, 1456)
0:49:47.293387
riverdischarge_yearY1998M01V01
(699, 1456)
0:49:51.392057
riverdischarge_yearY1999M01V01
(700, 1456)
0:49:55.565689
riverdischarge_yearY2000M01V01
(701, 1456)
0:49:59.772579
riverdischarge_yearY2001M01V01
(702, 1456)
0:50:03.515652
riverdischarge_yearY2002M01V01
(703, 1456)
0:50:07.635555
riverdischarge_yearY2003M01V01
(704, 1456)
0:50:11.914343
riverdischarge_yearY2004M01V01
(705, 1456)
0:50:16.079928
riverdischarge_yearY2005M01V01
(706, 1456)
0:50:20.267867
riverdischarge_yearY2006M01V01
(707, 1456)
0:50:24.518559
riverdischarge_yearY2007M01V01
(708, 1456)
0:50:28.668665
riverdischarge_yearY2008M01V01
(709, 1456)
0:50:32.819815
riverdischarge_yearY2009M01V01
(710, 1456)
0:50:37.037952
riverdischarge_yearY2010M01V01
(711, 1456)
0:50:41.205961
riverdischarge_yearY2011M01V01
(712, 1456)
0:50:45.405750
riverdischarge_yearY2012M01V01
(713, 1456)
0:50:49.609599
riverdischarge_yearY2013M01V01
(714, 1456)
0:50:53.726620
riverdischarge_yearY2014M01V01
(715, 1456)
0:

runoff_monthY1969M10V01
(857, 1456)
1:00:58.722656
runoff_monthY1969M11V01
(858, 1456)
1:01:02.481574
runoff_monthY1969M12V01
(859, 1456)
1:01:06.847806
runoff_monthY1970M01V01
(860, 1456)
1:01:11.079808
runoff_monthY1970M02V01
(861, 1456)
1:01:15.240765
runoff_monthY1970M03V01
(862, 1456)
1:01:19.657794
runoff_monthY1970M04V01
(863, 1456)
1:01:23.900797
runoff_monthY1970M05V01
(864, 1456)
1:01:28.081175
runoff_monthY1970M06V01
(865, 1456)
1:01:32.304975
runoff_monthY1970M07V01
(866, 1456)
1:01:36.491475
runoff_monthY1970M08V01
(867, 1456)
1:01:40.246558
runoff_monthY1970M09V01
(868, 1456)
1:01:44.079678
runoff_monthY1970M10V01
(869, 1456)
1:01:47.809580
runoff_monthY1970M11V01
(870, 1456)
1:01:52.033570
runoff_monthY1970M12V01
(871, 1456)
1:01:56.363768
runoff_monthY1971M01V01
(872, 1456)
1:02:00.221449
runoff_monthY1971M02V01
(873, 1456)
1:02:04.444504
runoff_monthY1971M03V01
(874, 1456)
1:02:08.119877
runoff_monthY1971M04V01
(875, 1456)
1:02:11.885965
runoff_monthY1971M05V01
(876, 1

(1017, 1456)
1:12:28.117968
runoff_monthY1983M03V01
(1018, 1456)
1:12:32.253572
runoff_monthY1983M04V01
(1019, 1456)
1:12:36.437716
runoff_monthY1983M05V01
(1020, 1456)
1:12:40.702505
runoff_monthY1983M06V01
(1021, 1456)
1:12:44.937365
runoff_monthY1983M07V01
(1022, 1456)
1:12:49.500759
runoff_monthY1983M08V01
(1023, 1456)
1:12:53.791285
runoff_monthY1983M09V01
(1024, 1456)
1:12:57.992313
runoff_monthY1983M10V01
(1025, 1456)
1:13:02.196317
runoff_monthY1983M11V01
(1026, 1456)
1:13:06.424898
runoff_monthY1983M12V01
(1027, 1456)
1:13:10.672749
runoff_monthY1984M01V01
(1028, 1456)
1:13:14.872472
runoff_monthY1984M02V01
(1029, 1456)
1:13:19.020409
runoff_monthY1984M03V01
(1030, 1456)
1:13:23.286465
runoff_monthY1984M04V01
(1031, 1456)
1:13:27.511975
runoff_monthY1984M05V01
(1032, 1456)
1:13:31.688312
runoff_monthY1984M06V01
(1033, 1456)
1:13:35.466839
runoff_monthY1984M07V01
(1034, 1456)
1:13:39.663577
runoff_monthY1984M08V01
(1035, 1456)
1:13:43.914450
runoff_monthY1984M09V01
(1036, 1456)

(1175, 1456)
1:23:40.945424
runoff_monthY1996M05V01
(1176, 1456)
1:23:44.214405
runoff_monthY1996M06V01
(1177, 1456)
1:23:48.423925
runoff_monthY1996M07V01
(1178, 1456)
1:23:52.575580
runoff_monthY1996M08V01
(1179, 1456)
1:23:56.840292
runoff_monthY1996M09V01
(1180, 1456)
1:24:01.035987
runoff_monthY1996M10V01
(1181, 1456)
1:24:05.518141
runoff_monthY1996M11V01
(1182, 1456)
1:24:09.745313
runoff_monthY1996M12V01
(1183, 1456)
1:24:13.915751
runoff_monthY1997M01V01
(1184, 1456)
1:24:18.048057
runoff_monthY1997M02V01
(1185, 1456)
1:24:22.277371
runoff_monthY1997M03V01
(1186, 1456)
1:24:26.472066
runoff_monthY1997M04V01
(1187, 1456)
1:24:30.678833
runoff_monthY1997M05V01
(1188, 1456)
1:24:34.488803
runoff_monthY1997M06V01
(1189, 1456)
1:24:38.257115
runoff_monthY1997M07V01
(1190, 1456)
1:24:42.441357
runoff_monthY1997M08V01
(1191, 1456)
1:24:46.642686
runoff_monthY1997M09V01
(1192, 1456)
1:24:50.824131
runoff_monthY1997M10V01
(1193, 1456)
1:24:55.053750
runoff_monthY1997M11V01
(1194, 1456)

(1333, 1456)
1:34:54.439249
runoff_monthY2009M07V01
(1334, 1456)
1:34:58.638386
runoff_monthY2009M08V01
(1335, 1456)
1:35:02.730156
runoff_monthY2009M09V01
(1336, 1456)
1:35:07.301545
runoff_monthY2009M10V01
(1337, 1456)
1:35:11.511580
runoff_monthY2009M11V01
(1338, 1456)
1:35:15.163883
runoff_monthY2009M12V01
(1339, 1456)
1:35:19.721737
runoff_monthY2010M01V01
(1340, 1456)
1:35:23.010424
runoff_monthY2010M02V01
(1341, 1456)
1:35:27.107328
runoff_monthY2010M03V01
(1342, 1456)
1:35:31.224561
runoff_monthY2010M04V01
(1343, 1456)
1:35:35.401418
runoff_monthY2010M05V01
(1344, 1456)
1:35:39.557900
runoff_monthY2010M06V01
(1345, 1456)
1:35:44.201914
runoff_monthY2010M07V01
(1346, 1456)
1:35:48.851076
runoff_monthY2010M08V01
(1347, 1456)
1:35:53.002819
runoff_monthY2010M09V01
(1348, 1456)
1:35:57.313682
runoff_monthY2010M10V01
(1349, 1456)
1:36:01.422923
runoff_monthY2010M11V01
(1350, 1456)
1:36:05.683269
runoff_monthY2010M12V01
(1351, 1456)
1:36:09.916115
runoff_monthY2011M01V01
(1352, 1456)

In [17]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

1:43:50.565734
